In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


#!pip3 install -q ktrain

In [2]:
import pandas as pd
from tqdm import tqdm_notebook
import os
from xlrd import *
import csv 
import sys
import win32com.client as w3c
from tempfile import NamedTemporaryFile

In [3]:
PATH = 'C:/Users/smsoh/Documents/GitHub/CLEF2021/task3/data/Task3a_Batch_1.xlsx'
PASS = 'fake@2021'

In [4]:
xlapp = w3c.Dispatch("Excel.Application")
xlwb = xlapp.Workbooks.Open(PATH, False, True, None, PASS)

In [5]:
#train = pd.read_csv("data/Task3a_Batch_1.xlsx")
f = NamedTemporaryFile(delete=False, suffix='.csv')
f.close()
os.unlink(f.name)

xlCSVWindows = 0x17
xlwb.SaveAs(Filename=f.name, FileFormat=xlCSVWindows)
train = pd.read_csv(f.name)

In [6]:
train

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,partially false
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,partially false
3,f14e8eb6,But things took a turn for the worse when riot...,Obama�s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It�s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff�s 78 Visits ...,false
...,...,...,...,...
432,e01f532a,Rosemary Frei and Patrick Corbett A high-prof...,�No one has died from the coronavirus�,false
433,3c791c29,As the coronavirus pandemic continues to devas...,Why Did 21 Million Phone Numbers Disappear fro...,false
434,fe6a9262,Belgium Health Minister Maggie de Block has pu...,Belgium Health Minister puts ban on non-essent...,false
435,97113f55,"Responding To Backlash, Netflix Clarifies Its ...","Responding To Backlash, Netflix Clarifies Its ...",false


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test = train_test_split(train, test_size=0.18, random_state=42)
X_val, X_test = train_test_split(X_test, test_size=0.5, random_state=42)

In [7]:
print('train size:',X_train.shape,',validation size:',X_val.shape,',test size:',X_test.shape)

train size: (41, 4) ,validation size: (4, 4) ,test size: (5, 4)


In [8]:
x_train = X_train["text"].tolist()
y_train = X_train["our rating"].tolist()
x_val = X_val["text"].tolist()
y_val = X_val["our rating"].tolist()
x_test = X_test["text"].tolist()
y_test = X_test["our rating"].tolist()

In [9]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-large-uncased'

In [10]:
t = text.Transformer(MODEL_NAME, maxlen=500, classes=[0,1])

C:\Users\smsoh\AppData\Local\Programs\Python\Python38\lib\site-packages\ktrain\text\preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


In [11]:
trn = t.preprocess_train(x_train, y_train)

preprocessing train...
language: en
train sequence lengths:
	mean : 602
	95percentile : 1951
	99percentile : 2047
C:\Users\smsoh\AppData\Local\Programs\Python\Python38\lib\site-packages\ktrain\utils.py:589: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


In [12]:
val = t.preprocess_train(x_val, y_val)

preprocessing train...
language: en
train sequence lengths:
	mean : 537
	95percentile : 1298
	99percentile : 1436


In [13]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)

In [15]:
learner.fit_onecycle(0.01, 10)



begin training using onecycle policy with max lr of 0.01...
Epoch 1/10


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
statement = 'Says Barack Obama complained that Democrats continue to ignore blacks except at election time.'
description = 'Great America Alliance said that Obama complained that Democrats continue to ignore blacks except at election time. \
              Obama did say the words used in the ad, but they weren’t his.He was quoting a Chicago barber talking about the days before African-Americans in Chicago helped elect the city’s first black mayor. \
              The full text from Obama’s book shows that he was arguing that voting can make a difference. Nothing in the section suggests that he thinks Democrats today only pay attention to blacks at election time.'
test = statement + '<>' + description
print(test)

In [ ]:
if predictor.predict(test) is 0: 
  print('False')  # indeed it is false
else:
  print('True')

In [ ]:
predictor.predict_proba(test)

In [ ]:
predictor.get_classes()

In [ ]:
pred = predictor.predict(x_test)

In [ ]:
j = 0
s = 0
for i in pred:
  if(i==y_test[j]):
    s+=1
  j+=1
print('The Accuracy of the test set is',s/2083*100)

In [ ]:
predictor.save('my_distilbert_predictor')SS

In [ ]:
reloaded_predictor = ktrain.load_predictor('my_distilbert_predictor')

Prediction

In [ ]:
#@title Prediction
statement = 'some person in campus has reported symptom of COVID 19 virus. ' #@param {type:"string"}
description = 'So far no person (student / faculty member / staff) in the campus has reported any symptom of COVID 19 virus. ' #@param {type:"string"}
test = statement + '<>' + description
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Enter")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    if reloaded_predictor.predict(test) is 0: 
      print('False')  
    else:
      print('True')

button.on_click(on_button_clicked)
display(button, output)